# Training Models with the Data-Driven Library

The datadriven library provides an extensible command-line interface for training, evaluating, and predicting data-driven simulators. However, you may prefer training and sweeping models inside a notebook. This notebook provides an example for doing so.

## Set Working Directory and Import Necessary Libraries

In [1]:
cd ..

/Users/jill/bonsai/datadrivenmodel


In [2]:
from hydra.experimental import initialize, compose
from omegaconf import DictConfig, ListConfig, OmegaConf
from model_loader import available_models
from base import plot_parallel_coords
import logging
import matplotlib.pyplot as plt
import numpy as np
from rich import print
from rich.logging import RichHandler
import copy
import pandas as pd
from assessment_metrics_loader import available_metrics

logging.basicConfig(
    level=logging.INFO,
    format="%(message)s",
    datefmt="[%X]",
    handlers=[RichHandler()]
)
logger = logging.getLogger("ddm_training")
logger.setLevel(logging.INFO)

## Initialize Configuration

While you can provide every argument manually, there is benefit in directly using the `hydra` config class to load an existing configuration file. This way you can ensure your parameters are saved to a file for later use, and you automatically gain the benefit of all the logging and model artifacts that are provided by our workflow of `hydra` and `mlflow`.

If you want to override any settings of the configurations, provide them in a list of `overrides` as shown below.

In [3]:
initialize(config_path="../conf", job_name="ddm_training")
cfg = compose(config_name="config", overrides=["data=cartpole-updated", "model=xgboost"])

/Users/jill/opt/anaconda3/envs/ddm/lib/python3.8/site-packages/hydra/plugins/config_source.py:190: UserWarning: 
Missing @package directive model/xgboost.yaml in file:///Users/jill/bonsai/datadrivenmodel/conf.
See https://hydra.cc/docs/next/upgrades/0.11_to_1.0/adding_a_package_directive
  warnings.warn(message=msg, category=UserWarning)
/Users/jill/opt/anaconda3/envs/ddm/lib/python3.8/site-packages/hydra/plugins/config_source.py:190: UserWarning: 
Missing @package directive data/cartpole_st1_at.yaml in file:///Users/jill/bonsai/datadrivenmodel/conf.
See https://hydra.cc/docs/next/upgrades/0.11_to_1.0/adding_a_package_directive
  warnings.warn(message=msg, category=UserWarning)
/Users/jill/opt/anaconda3/envs/ddm/lib/python3.8/site-packages/hydra/plugins/config_source.py:190: UserWarning: 
Missing @package directive simulator/house_energy_simparam.yaml in file:///Users/jill/bonsai/datadrivenmodel/conf.
See https://hydra.cc/docs/next/upgrades/0.11_to_1.0/adding_a_package_directive
  warn

In [4]:
print(OmegaConf.to_yaml(cfg))

In [5]:
# Extract features from yaml file
input_cols = cfg['data']['inputs']
output_cols = cfg['data']['outputs']
augmented_cols = cfg['data']['augmented_cols']
dataset_path = cfg['data']['path']
iteration_order = cfg['data']['iteration_order']
episode_col = cfg['data']['episode_col']
iteration_col = cfg['data']['iteration_col']
max_rows = cfg['data']['max_rows']
test_perc = cfg['data']['test_perc']

## Model Trainer

To make it easy to sweep over models later, we create a simple `train_models` function here:

In [6]:
def train_models(config=cfg):

    logger.info(f'Model type: {available_models[config["model"]["name"]]}')
    Model = available_models[config["model"]["name"]]
    model = Model()
    logger.info(f"Building model with parameters: {config}")
    model.build_model(
        **config["model"]["build_params"]
    )
    logger.info(f"Loading data from {dataset_path}")
    X, y = model.load_csv(
        input_cols=input_cols,
        output_cols=output_cols,
        augm_cols=list(augmented_cols),
        dataset_path=dataset_path,
        iteration_order=iteration_order,
        episode_col=episode_col,
        iteration_col=iteration_col,
        max_rows=max_rows,
    )
    global X_train, y_train, episode_ids_train, X_test, y_test, episode_ids_test
    train_id_end = int(np.floor(X.shape[0] * (1 - test_perc)))
    X_train, y_train, episode_ids_train = (X[:train_id_end,],y[:train_id_end,],model.episode_ids[:train_id_end,])
    X_test, y_test, episode_ids_test = (X[train_id_end:,],y[train_id_end:,],model.episode_ids[train_id_end:,])
    
    
    logger.info(f"Fitting model...")
    model.fit(X_train, y_train)
    logger.info(f"Model trained!")
    y_pred = model.predict(X_test)
    r2_score = available_metrics["r2_score"]
    logger.info(f"R^2 score is {r2_score(y_test,y_pred)} for the test set.")

    return model

In [7]:
model = train_models(cfg)

## Hyperparameter Sweeping

The `datadrivenmodel` has an automatic solution for hyperparameter sweeping and tuning. These settings are provided in the config `model.sweep` parameters. Provide the limits of the variables you want to sweep over and the `sweep` method will automatically parallelize the sweep over the available number of cores and find the optimal solution according to your `scoring_func`.

### Configuration Parameters

You can select the search algorithm you'd like to use: `bayesian` runs bayesian optimiziation (using scikit-optimize), `hyperopt` runs [Tree-Parzen Estimators](https://papers.nips.cc/paper/2011/file/86e8f7ab32cfd12577bc2619bc635690-Paper.pdf) with the `hyperopt` package, `bohb` uses Bayesian Opt/HyperBand, or `optuna` which also runs Tree-Parzen estimators but using the [`optuna`](https://optuna.readthedocs.io/en/stable/) package.

In [8]:
print(OmegaConf.to_yaml(cfg["model"]["sweep"]))

In [10]:
params = OmegaConf.to_container(cfg["model"]["sweep"]["params"])
logger.info(f"Sweeping with parameters: {params}")

sweep_df = model.sweep(
    params=params,
    X=X_train,
    y=y_train,
    search_algorithm=cfg["model"]["sweep"]["search_algorithm"],
    num_trials=cfg["model"]["sweep"]["num_trials"],
    scoring_func=cfg["model"]["sweep"]["scoring_func"],
    results_csv_path=cfg["model"]["sweep"]["results_csv_path"],
)    

(raylet) Traceback (most recent call last):
(raylet)   File "/Users/jill/opt/anaconda3/envs/ddm/lib/python3.8/site-packages/ray/new_dashboard/agent.py", line 334, in <module>
(raylet)     raise e
(raylet)   File "/Users/jill/opt/anaconda3/envs/ddm/lib/python3.8/site-packages/ray/new_dashboard/agent.py", line 323, in <module>
(raylet)     loop.run_until_complete(agent.run())
(raylet)   File "/Users/jill/opt/anaconda3/envs/ddm/lib/python3.8/asyncio/base_events.py", line 616, in run_until_complete
(raylet)     return future.result()
(raylet)   File "/Users/jill/opt/anaconda3/envs/ddm/lib/python3.8/site-packages/ray/new_dashboard/agent.py", line 121, in run
(raylet)     self.aioredis_client = await dashboard_utils.get_aioredis_client(
(raylet)   File "/Users/jill/opt/anaconda3/envs/ddm/lib/python3.8/site-packages/ray/new_dashboard/utils.py", line 662, in get_aioredis_client
(raylet)     return await aioredis.create_redis_pool(
(raylet) AttributeError: module 'aioredis' has no attribute 'cr

(raylet) Traceback (most recent call last):
(raylet)   File "/Users/jill/opt/anaconda3/envs/ddm/lib/python3.8/site-packages/ray/new_dashboard/agent.py", line 334, in <module>
(raylet)     raise e
(raylet)   File "/Users/jill/opt/anaconda3/envs/ddm/lib/python3.8/site-packages/ray/new_dashboard/agent.py", line 323, in <module>
(raylet)     loop.run_until_complete(agent.run())
(raylet)   File "/Users/jill/opt/anaconda3/envs/ddm/lib/python3.8/asyncio/base_events.py", line 616, in run_until_complete
(raylet)     return future.result()
(raylet)   File "/Users/jill/opt/anaconda3/envs/ddm/lib/python3.8/site-packages/ray/new_dashboard/agent.py", line 121, in run
(raylet)     self.aioredis_client = await dashboard_utils.get_aioredis_client(
(raylet)   File "/Users/jill/opt/anaconda3/envs/ddm/lib/python3.8/site-packages/ray/new_dashboard/utils.py", line 662, in get_aioredis_client
(raylet)     return await aioredis.create_redis_pool(
(raylet) AttributeError: module 'aioredis' has no attribute 'cr

(raylet) Traceback (most recent call last):
(raylet)   File "/Users/jill/opt/anaconda3/envs/ddm/lib/python3.8/site-packages/ray/new_dashboard/agent.py", line 334, in <module>
(raylet)     raise e
(raylet)   File "/Users/jill/opt/anaconda3/envs/ddm/lib/python3.8/site-packages/ray/new_dashboard/agent.py", line 323, in <module>
(raylet)     loop.run_until_complete(agent.run())
(raylet)   File "/Users/jill/opt/anaconda3/envs/ddm/lib/python3.8/asyncio/base_events.py", line 616, in run_until_complete
(raylet)     return future.result()
(raylet)   File "/Users/jill/opt/anaconda3/envs/ddm/lib/python3.8/site-packages/ray/new_dashboard/agent.py", line 121, in run
(raylet)     self.aioredis_client = await dashboard_utils.get_aioredis_client(
(raylet)   File "/Users/jill/opt/anaconda3/envs/ddm/lib/python3.8/site-packages/ray/new_dashboard/utils.py", line 662, in get_aioredis_client
(raylet)     return await aioredis.create_redis_pool(
(raylet) AttributeError: module 'aioredis' has no attribute 'cr

(raylet) Traceback (most recent call last):
(raylet)   File "/Users/jill/opt/anaconda3/envs/ddm/lib/python3.8/site-packages/ray/new_dashboard/agent.py", line 334, in <module>
(raylet)     raise e
(raylet)   File "/Users/jill/opt/anaconda3/envs/ddm/lib/python3.8/site-packages/ray/new_dashboard/agent.py", line 323, in <module>
(raylet)     loop.run_until_complete(agent.run())
(raylet)   File "/Users/jill/opt/anaconda3/envs/ddm/lib/python3.8/asyncio/base_events.py", line 616, in run_until_complete
(raylet)     return future.result()
(raylet)   File "/Users/jill/opt/anaconda3/envs/ddm/lib/python3.8/site-packages/ray/new_dashboard/agent.py", line 121, in run
(raylet)     self.aioredis_client = await dashboard_utils.get_aioredis_client(
(raylet)   File "/Users/jill/opt/anaconda3/envs/ddm/lib/python3.8/site-packages/ray/new_dashboard/utils.py", line 662, in get_aioredis_client
(raylet)     return await aioredis.create_redis_pool(
(raylet) AttributeError: module 'aioredis' has no attribute 'cr

(raylet) Traceback (most recent call last):
(raylet)   File "/Users/jill/opt/anaconda3/envs/ddm/lib/python3.8/site-packages/ray/new_dashboard/agent.py", line 334, in <module>
(raylet)     raise e
(raylet)   File "/Users/jill/opt/anaconda3/envs/ddm/lib/python3.8/site-packages/ray/new_dashboard/agent.py", line 323, in <module>
(raylet)     loop.run_until_complete(agent.run())
(raylet)   File "/Users/jill/opt/anaconda3/envs/ddm/lib/python3.8/asyncio/base_events.py", line 616, in run_until_complete
(raylet)     return future.result()
(raylet)   File "/Users/jill/opt/anaconda3/envs/ddm/lib/python3.8/site-packages/ray/new_dashboard/agent.py", line 121, in run
(raylet)     self.aioredis_client = await dashboard_utils.get_aioredis_client(
(raylet)   File "/Users/jill/opt/anaconda3/envs/ddm/lib/python3.8/site-packages/ray/new_dashboard/utils.py", line 662, in get_aioredis_client
(raylet)     return await aioredis.create_redis_pool(
(raylet) AttributeError: module 'aioredis' has no attribute 'cr

...

(raylet) Traceback (most recent call last):
(raylet)   File "/Users/jill/opt/anaconda3/envs/ddm/lib/python3.8/site-packages/ray/new_dashboard/agent.py", line 334, in <module>
(raylet)     raise e
(raylet)   File "/Users/jill/opt/anaconda3/envs/ddm/lib/python3.8/site-packages/ray/new_dashboard/agent.py", line 323, in <module>
(raylet)     loop.run_until_complete(agent.run())
(raylet)   File "/Users/jill/opt/anaconda3/envs/ddm/lib/python3.8/asyncio/base_events.py", line 616, in run_until_complete
(raylet)     return future.result()
(raylet)   File "/Users/jill/opt/anaconda3/envs/ddm/lib/python3.8/site-packages/ray/new_dashboard/agent.py", line 121, in run
(raylet)     self.aioredis_client = await dashboard_utils.get_aioredis_client(
(raylet)   File "/Users/jill/opt/anaconda3/envs/ddm/lib/python3.8/site-packages/ray/new_dashboard/utils.py", line 662, in get_aioredis_client
(raylet)     return await aioredis.create_redis_pool(
(raylet) AttributeError: module 'aioredis' has no attribute 'cr


(raylet)   File "/Users/jill/opt/anaconda3/envs/ddm/lib/python3.8/site-packages/ray/new_dashboard/agent.py", line 334, in <module>
(raylet)     raise e
(raylet)   File "/Users/jill/opt/anaconda3/envs/ddm/lib/python3.8/site-packages/ray/new_dashboard/agent.py", line 323, in <module>
(raylet)     loop.run_until_complete(agent.run())
(raylet)   File "/Users/jill/opt/anaconda3/envs/ddm/lib/python3.8/asyncio/base_events.py", line 616, in run_until_complete
(raylet)     return future.result()
(raylet)   File "/Users/jill/opt/anaconda3/envs/ddm/lib/python3.8/site-packages/ray/new_dashboard/agent.py", line 121, in run
(raylet)     self.aioredis_client = await dashboard_utils.get_aioredis_client(
(raylet)   File "/Users/jill/opt/anaconda3/envs/ddm/lib/python3.8/site-packages/ray/new_dashboard/utils.py", line 662, in get_aioredis_client
(raylet)     return await aioredis.create_redis_pool(
(raylet) AttributeError: module 'aioredis' has no attribute 'create_redis_pool'
2021/11/18 11:36:31 WARNIN

2021/11/18 11:36:32 WARNING mlflow.sklearn: RegressorMixin.score failed. The 'training_score' metric will not be recorded. Scoring error: 'GaussianProcessRegressor' object has no attribute 'y_train_std_'
2021/11/18 11:36:32 WARNING mlflow.sklearn.utils: Failed to autolog metrics for GaussianProcessRegressor. Logging error: 'GaussianProcessRegressor' object has no attribute 'y_train_std_'
2021/11/18 11:36:32 WARNING mlflow.sklearn: Failed to infer model signature: 'GaussianProcessRegressor' object has no attribute 'y_train_std_'
2021/11/18 11:36:32 WARNING mlflow.sklearn: RegressorMixin.score failed. The 'training_score' metric will not be recorded. Scoring error: 'GaussianProcessRegressor' object has no attribute 'y_train_std_'
2021/11/18 11:36:32 WARNING mlflow.sklearn.utils: Failed to autolog metrics for GaussianProcessRegressor. Logging error: 'GaussianProcessRegressor' object has no attribute 'y_train_std_'
2021/11/18 11:36:32 WARNING mlflow.sklearn: Failed to infer model signature

2021/11/18 11:36:33 WARNING mlflow.sklearn.utils: Truncated the value of the key `estimator`. Truncated value: `XGBRegressor(base_score=None, booster=None, colsample_bylevel=None,
             colsample_bynode=None, colsample_bytree=None, eta=0.3, gamma=0.5,
             gpu_id=None, importance_type='gain', interaction_constraints=None,
             learnin...`


(raylet) Traceback (most recent call last):
(raylet)   File "/Users/jill/opt/anaconda3/envs/ddm/lib/python3.8/site-packages/ray/new_dashboard/agent.py", line 334, in <module>
(raylet)     raise e
(raylet)   File "/Users/jill/opt/anaconda3/envs/ddm/lib/python3.8/site-packages/ray/new_dashboard/agent.py", line 323, in <module>
(raylet)     loop.run_until_complete(agent.run())
(raylet)   File "/Users/jill/opt/anaconda3/envs/ddm/lib/python3.8/asyncio/base_events.py", line 616, in run_until_complete
(raylet)     return future.result()
(raylet)   File "/Users/jill/opt/anaconda3/envs/ddm/lib/python3.8/site-packages/ray/new_dashboard/agent.py", line 121, in run
(raylet)     self.aioredis_client = await dashboard_utils.get_aioredis_client(
(raylet)   File "/Users/jill/opt/anaconda3/envs/ddm/lib/python3.8/site-packages/ray/new_dashboard/utils.py", line 662, in get_aioredis_client
(raylet)     return await aioredis.create_redis_pool(
(raylet) AttributeError: module 'aioredis' has no attribute 'cr

In [ ]:
sweep_df.head()

## Results and Outputs

All outputs are saved to a timestamped directory in `outputs`. This includes the model artifacts, hyperparameter tuning results, and a verbose log of the entire run.

In [ ]:
ls -lh outputs/

In [ ]:
!tree -L 3 outputs/2021-04-21/

### Visualizing Hyperparameter Results

In [ ]:
plot_parallel_coords(sweep_df)

### Reading Saved Runs from CSV

Runs are automatically saved to a CSV in the outputs directory:

In [ ]:
sweep_df2 = pd.read_csv("outputs/2021-04-21/10-29-25/xgboost_gridsearch/search_results.csv")

In [ ]:
plot_parallel_coords(sweep_df2)